## Import Libraries
In order to generate a bag of words representation we need the following libraries

In [2]:
import numpy as np
import json
import pandas as pd

## Import Data


In [9]:
data_set = np.array([ [x['venue'], x['citations']] for x in json.load(open('train-1.json'))[:10] if x['venue']])
# data_set = np.array([ [x['venue'], x['citations']] for x in json.load(open('train-1.json')) if x['venue']])
# print(data_set[0])

## Proportion analysis of journals/venues
Aim of the code is to divide the data in n equal portions, where the 'lowest' division accounts for the group of X that contributes the least to a given Y.

In [10]:
def get_cat(dataFrame):
    conditions = [
        (dataFrame['cumsum'] == 0),
        (dataFrame['cumsum'] > 0) & (dataFrame['cumsum'] <= 25),
        (dataFrame['cumsum'] > 25) & (dataFrame['cumsum'] <= 50),
        (dataFrame['cumsum'] > 50) & (dataFrame['cumsum'] <= 75),
        (dataFrame['cumsum'] >= 75)
    ]
    values = [1, 2, 3, 4,5]
    dataFrame['class_Venue'] = np.select(conditions, values)
    return dataFrame

def get_venue_dictionary(XY):
    #split data 
    X, _ = np.split(XY, 2, 1)
    
    unique_X = np.unique(X,return_counts=True)
    unique_X_length = len(unique_X[0])
    print(unique_X_length)
    zero_array = np.zeros((3,unique_X_length))

    merge = [unique_X[0], unique_X[1].astype(int), zero_array[0].astype(int), zero_array[1].astype(int), zero_array[2].astype(float)]
    
    # Compute citations per topic
    for x in XY :
        merge[2][np.where(merge[0] == x[0])[0][0]] += int(x[1])

    # Compute citations per topic divided by the amount of articles
    for (i, j) in enumerate(merge[3]):
        merge[3][i] = (merge[2][i]/merge[1][i])

    # Compute new summed citations
    summed_citations = np.sum(merge[3])
    result = []
    for (i, j) in enumerate(merge[3]):
        merge[4][i] = 100/summed_citations*merge[3][i]
        result.append((merge[0][i],merge[1][i],merge[2][i],merge[3][i],merge[4][i]))
    
    dtype = [('venue', 'S100'), ('count', int), ('summed_citations', int), ('average_citations', float), ('contribution', float)]
    
    structured_array = np.array(result, dtype=dtype)
    sorted = np.sort(structured_array, order='contribution')
    df = pd.DataFrame(sorted)
    df['cumsum'] = df['contribution'].cumsum(axis=0)
    df = get_cat(df)
    # return {x['field_of_study'] : x['class_Venue'] for x in df}
    return df
    # return (df['field_of_study'],df['class_Venue'])
    
get_venue_dictionary(data_set).to_pickle("venue_data.pkl")
get_venue_dictionary(data_set)

9
9


,venue,count,summed_citations,average_citations,contribution,cumsum,class_Venue
0,b'ACL/IJCNLP',1,0,0.0,0.000000,0.000000,1
1,b'CoNLL',1,0,0.0,0.000000,0.000000,1
2,b'EVAL4NLP',1,1,1.0,1.075269,1.075269,2
3,b'CoNLL Shared Task',1,4,4.0,4.301075,5.376344,2
4,b'INLG',1,5,5.0,5.376344,10.752688,2
5,b'SemEval@ACL',1,5,5.0,5.376344,16.129032,2
6,b'EACL 2014',1,8,8.0,8.602151,24.731183,2
7,b'IWPT',1,10,10.0,10.752688,35.483871,3
8,b'ACL',1,60,60.0,64.516129,100.000000,5
